In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import squarify

# Data processing, metrics and modeling
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix,  roc_curve, precision_recall_curve, accuracy_score, roc_auc_score
import lightgbm as lgbm
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from yellowbrick.classifier import DiscriminationThreshold

# Stats
import scipy.stats as ss
from scipy import interp
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# Time
from contextlib import contextmanager
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Who are Pima Indians ?

"The Pima (or Akimel O'odham, also spelled Akimel O'otham, "River People", formerly known as Pima) are a group of Native Americans living in an area consisting of what is now central and southern Arizona. The majority population of the surviving two bands of the Akimel O'odham are based in two reservations: the Keli Akimel O'otham on the Gila River Indian Community (GRIC) and the On'k Akimel O'odham on the Salt River Pima-Maricopa Indian Community (SRPMIC)." Wikipedia

![](https://cdn.britannica.com/42/93542-050-E2B32DAB/women-Pima-shinny-game-field-hockey.jpg)

# <a id='1'>1. Load Libraries and read the data<a>

## <a id='1.1'>1.1. Load libraries Pandas, Numpy and Matplotlib</a> 

In [ ]:
# Python libraries
# Classic,data manipulation and linear algebra
import pandas as pd
import numpy as np

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


## <a id='1.2'>1.2 Read Data </a>

In [ ]:
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

# <a id='2'>2. Overview of each Attribute and its Characteristics</a>

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.



## <a id='2.1'>2.1 Head and  Attributes</a>

In [ ]:
data.head()

## What is diabetes ?
Acccording to NIH, "Diabetes is a disease that occurs when your blood glucose, also called blood sugar, is too high. Blood glucose is your main source of energy and comes from the food you eat. Insulin, a hormone made by the pancreas, helps glucose from food get into your cells to be used for energy. Sometimes your body doesn’t make enough—or any—insulin or doesn’t use insulin well. Glucose then stays in your blood and doesn’t reach your cells.

Over time, having too much glucose in your blood can cause health problems. Although diabetes has no cure, you can take steps to manage your diabetes and stay healthy.

Sometimes people call diabetes “a touch of sugar” or “borderline diabetes.” These terms suggest that someone doesn’t really have diabetes or has a less serious case, but every case of diabetes is serious.

**What are the different types of diabetes?** The most common types of diabetes are type 1, type 2, and gestational diabetes.

**Type 1 diabetes** If you have type 1 diabetes, your body does not make insulin. Your immune system attacks and destroys the cells in your pancreas that make insulin. Type 1 diabetes is usually diagnosed in children and young adults, although it can appear at any age. People with type 1 diabetes need to take insulin every day to stay alive.

**Type 2 diabetes** If you have type 2 diabetes, your body does not make or use insulin well. You can develop type 2 diabetes at any age, even during childhood. However, this type of diabetes occurs most often in middle-aged and older people. Type 2 is the most common type of diabetes.

**Gestational diabetes** Gestational diabetes develops in some women when they are pregnant. Most of the time, this type of diabetes goes away after the baby is born. However, if you’ve had gestational diabetes, you have a greater chance of developing type 2 diabetes later in life. Sometimes diabetes diagnosed during pregnancy is actually type 2 diabetes.

**Other types of diabetes** Less common types include monogenic diabetes, which is an inherited form of diabetes, and cystic fibrosis-related diabetes ."

### What are these columns signifying?

This datasets consist of eight medical predictor (independent) variables and one target (dependent) variable, Outcome Class variable whether paitent has diabetes or not (encoded as 0 or 1). Predictor variables are  includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.
* Pregnancies - Number of times Pregnant. 
* Glucose - Plasma glucose concentration a 2 hours in an oral glucose tolerance test. 
* BloodPressure - Diastolic blood pressure (mm Hg).
* SkinThickness - Triceps skin fold thickness (mm).
* Insulin - 2-Hour serum insulin (mu U/ml).
* BMI - Body mass index ( BMI weight in kg/(height in m)^2).
* DiabetesPedigreeFunction - Diabetes pedigree function.
* Age - Age in year.
* Outcome - 1 if Patient has diabetes, 0 otherwise.

### What is DiabetesPedigreeFunction Column signifying?
A function which scores the likelihood of diabetes based on family history. It provided some data on diabetes mellitus history in relatives and the genetic relationship of those relatives to the patient.

## <a id='2.2'>2.2 Size and Type of Data</a>

In [ ]:
data.info()

The dataset has 768 entries and 9 colums(8 Predictor variables and 1 Target variable). Only BMI and DaiabetesPedigreeFunction are floating numbers, rest are integers.

In [ ]:
# 2 datasets
D = data[(data['Outcome'] != 0)]
H = data[(data['Outcome'] == 0)]

## <a id='2.3'>2.3 Missing Values</a>

Let's calculate and plot the Percentage of data missing in each attribute

In [ ]:
missing_values = (data.isnull().sum() / len(data) * 100).round(2)
missing_values = missing_values[missing_values > 0]
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(inplace=True)
missing_values

It looks like there's no data is missing, Hurray!
Is it?
We saw on data.head() that some features like BloodPressure contain 0, it doesn't make sense here and this indicates missing values.
Below we replace 0 value by NaN.

In [ ]:
data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [ ]:
missing_values = (data.isnull().sum() / len(data) * 100).round(2)
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(inplace=True)
missing_values

Attributes with Missing Values and percentage missing

- Glucose           0.65%
- BMI               1.43%
- BloodPressure     4.56%
- SkinThickness    29.56%
- Insulin          48.70%

In [ ]:
missing_values = missing_values.to_frame()
missing_values.columns = ['Percentage Missing']
missing_values.index.names = ['Attributes']
missing_values['Attributes'] = missing_values.index

In [ ]:
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

In [ ]:
sns.barplot(x = 'Attributes', y = 'Percentage Missing', data=missing_values)
plt.xticks(rotation = 90)
plt.show()

 ## <a id='2.4'> 2.4 Identify Noise in the Data and Type of Noise</a>

In [ ]:
data.describe()

Since all the data is in the form of integers and floating points we can draw histograms to see the data distribution 

 ## <a id='2.5'> 2.5 Data Visualization</a>

In [ ]:
data.hist(bins=25, figsize=(20, 15));

In [ ]:
plt.style.use('ggplot') # Using ggplot2 style visuals 

f, ax = plt.subplots(figsize=(10, 20))

ax.set_facecolor('#fafafa')
ax.set(xlim=(-1, 500))
plt.ylabel('Variables')
plt.title("Box Plot of Variables")
ax = sns.boxplot(data = data, 
  orient = 'h', 
  palette = 'Set2')

From the above information we can make the following obserevations for the attributes 

- The max Pregnacies is 17 which is quite an outlier.
- The Glucose data is normally distributed.
- Blood Pressure is also normally distributed with a considerable number of outliers but still is not skewed.
- Skin Thickness is positvely skewed with some outliers.
- Insulin also has considerable number of outliers with abnormal values around 800 and is positively skewed.
- BMI also has few outliers and is positively skewed.
- DiabetesPedigreeFunction is also highly positively skewed with few outliers.
- Age data has very few outliers.  


# <a id='3'> 3. Target</a> 

What's target's distribution ? 

In [ ]:
ax = sns.catplot(x="Outcome", kind="count", data=data).set(title = "Outcome Distribution");

# <a id='4'> 4. Correlations between the attributes</a>

In [ ]:
corr_matrix = data.corr()

In [ ]:
corr_matrix["Outcome"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes =  ['Outcome','Glucose', 'BMI', 'Insulin', 'SkinThickness', 'Age',  'Pregnancies', 'DiabetesPedigreeFunction', 'BloodPressure']

scatter_matrix(data[attributes], figsize=(20, 20));

### Correlation Heat Map

In [ ]:

fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(corr_matrix, annot=True);

# <a id='5'> 5. Data Cleaning & EDA </a>

## <a id='5.1'> 5.1 Handling Missing Data </a>

Many of the attributes have significant outliers so we shall replace the missing values with the median by target, instead of the mean.

In [ ]:
def median_target(var):   
    temp = data[data[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp

### <a id='5.1.a'> 5.1.a Insulin </a>

* **Insulin** : 2-Hour serum insulin (mu U/ml)

In [ ]:
def plot_distribution(data_select, size_bin) :  
    # 2 datasets
    tmp1 = D[data_select]
    tmp2 = H[data_select]
    hist_data = [tmp1, tmp2]
    
    group_labels = ['diabetic', 'healthy']
    colors = ['red', 'lime']

    fig = ff.create_distplot(hist_data, group_labels, colors = colors, show_hist = True, bin_size = size_bin, curve_type='kde')
    
    fig['layout'].update(title = data_select)

    py.iplot(fig, filename = 'Density plot')

In [ ]:
plot_distribution('Insulin', 0)

In [ ]:
median_target('Insulin')

Insulin's medians by the target are really different ! 102.5 for a healthy person and 169.5 for a diabetic person

In [ ]:
data.loc[(data['Outcome'] == 0 ) & (data['Insulin'].isnull()), 'Insulin'] = 102.5
data.loc[(data['Outcome'] == 1 ) & (data['Insulin'].isnull()), 'Insulin'] = 169.5

### <a id='5.1.b'> 5.1.b Glucose </a>

* **Glucose** : Plasma glucose concentration a 2 hours in an oral glucose tolerance test

In [ ]:
plot_distribution('Glucose', 0)

In [ ]:
median_target('Glucose')

In [ ]:
data.loc[(data['Outcome'] == 0 ) & (data['Glucose'].isnull()), 'Glucose'] = 107
data.loc[(data['Outcome'] == 1 ) & (data['Glucose'].isnull()), 'Glucose'] = 140

107 for a healthy person and 140 for a diabetic person

### <a id='5.1.c'> 5.1.c SkinThickness</a>

* ** SkinThickness** : Triceps skin fold thickness (mm)

In [ ]:
plot_distribution('SkinThickness', 10)

In [ ]:
median_target('SkinThickness')

In [ ]:
data.loc[(data['Outcome'] == 0 ) & (data['SkinThickness'].isnull()), 'SkinThickness'] = 27
data.loc[(data['Outcome'] == 1 ) & (data['SkinThickness'].isnull()), 'SkinThickness'] = 32

27 for a healthy person and 32 for a diabetic person

### <a id='5.1.d'>5.1.d BloodPressure</a> 

In [ ]:
plot_distribution('BloodPressure', 5)

In [ ]:
median_target('BloodPressure')

In [ ]:
data.loc[(data['Outcome'] == 0 ) & (data['BloodPressure'].isnull()), 'BloodPressure'] = 70
data.loc[(data['Outcome'] == 1 ) & (data['BloodPressure'].isnull()), 'BloodPressure'] = 74.5

## <a id='5.1.e'>5.1.e BMI</a> 

* **BMI** : Body mass index (weight in kg/(height in m)^2)

In [ ]:
plot_distribution('BMI', 0)

In [ ]:
median_target('BMI')

In [ ]:
data.loc[(data['Outcome'] == 0 ) & (data['BMI'].isnull()), 'BMI'] = 30.1
data.loc[(data['Outcome'] == 1 ) & (data['BMI'].isnull()), 'BMI'] = 34.3

* **Age** : Age (years)
* **DiabetesPedigreeFunction** : Diabetes pedigree function
* **Pregnancies** : Number of times pregnant

In [ ]:
#plot distribution 
plot_distribution('Age', 0)
plot_distribution('Pregnancies', 0)
plot_distribution('DiabetesPedigreeFunction', 0)

All features are filled the missing values. Now, we can create new features.

# <a id='6'> 6. Feature Engineering & Selection </a>

We need to add important features to the dataset discover some effective features before training the data to machine learning models.

**Feature 1 : BMI Descriptor** Adding BMI Descriptor feature as we know all the participants were adult female.
The features are encoded as below
**BMI**
* Under - Under 18.5 – are considered underweight and possibly malnourished.
* Healthy - 18.5 to 24.9 – are within a healthy weight range for young and middle-aged adults.
* OverWeight - 25.0 to 29.9 – are considered overweight.
* Obese - Over 30 – are considered obese.

In [ ]:
def set_bmi(x):
    if x < 18.5:
        return "Under Weight"
    elif x >= 18.5 and x <= 24.9:
        return "Healthy"
    elif x >= 25 and x <= 29.9:
        return "Over Weight"
    elif x >= 30:
        return "Obese"

In [ ]:
data["BMI_CAT"] = data.BMI.apply(set_bmi)
data.head()

**Feature 2**:
* Insulin Indicative Range If insulin level (2-Hour serum insulin (mu U/ml)) is >= 16 and <= 166, then it is **normal** range else it is considered as **Abnormal**.

In [ ]:
def set_insulin(x):
    # Normal
    if x >= 16 and x <= 166:
        return "Normal"
    # Abnormal
    else:
        return "Abnormal"

In [ ]:
data["INSULIN_CAT"] = data.Insulin.apply(set_insulin)
data.head()

In [ ]:
ax = sns.catplot(x="INSULIN_CAT", kind="count", data=data).set(title = "Insulin Levels Distribution");

In [ ]:
ax = sns.catplot(x="BMI_CAT", kind="count", data=data).set(title = "BMI Categorical Distribution");

# <a id='7'> 7. Data Preparation </a>

## <a id='7.1'> 7.1 Standard Scaler and Label Encoder </a>

* **StandardScaler** :

Standardize features by removing the mean and scaling to unit variance : 

![](https://i.stack.imgur.com/PZgJ2.png)

Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the set. Mean and standard deviation are then stored to be used on later data using the transform method.

Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual features do not more or less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance).

* **LabelEncoder** : Encode labels with value between 0 and n_classes-1.

Bellow we encode the data to feed properly to our algorithm

In [ ]:
target_col = ["Outcome"]
cat_cols = ["BMI_CAT", "INSULIN_CAT"]

#numeric columns
num_cols = [x for x in data.columns if x not in cat_cols + target_col]

#Binary columns with 2 values
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])
    
#Duplicating columns for multi value columns
data = pd.get_dummies(data = data,columns = multi_cols )

#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_data_og = data.copy()
data = data.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index=True,right_index=True,how = "left")

We can create a correlation matrix with new features

## <a id='7.2'> 7.2 Correlation Matrix </a>

A **correlation matrix** is a table showing correlation coefficients between sets of variables. Each random variable (Xi) in the table is correlated with each of the other values in the table (Xj). This allows you to see which pairs have the highest correlation.

In [ ]:
def correlation_plot():
    #correlation
    correlation = data.corr()
    #tick labels
    matrix_cols = correlation.columns.tolist()
    #convert to array
    corr_array  = np.array(correlation)
    trace = go.Heatmap(z = corr_array,
                       x = matrix_cols,
                       y = matrix_cols,
                       colorscale='Viridis',
                       colorbar   = dict() ,
                      )
    layout = go.Layout(dict(title = 'Correlation Matrix for variables',
                            #autosize = False,
                            #height  = 1400,
                            #width   = 1600,
                            margin  = dict(r = 0 ,l = 100,
                                           t = 0,b = 100,
                                         ),
                            yaxis   = dict(tickfont = dict(size = 9)),
                            xaxis   = dict(tickfont = dict(size = 9)),
                           )
                      )
    fig = go.Figure(data = [trace],layout = layout)
    py.iplot(fig)

In [ ]:
correlation_plot()

## <a id='7.3'>7.3. X and y</a> 

We define X and y:

In [ ]:
# Def X and Y
X = data.drop('Outcome', 1)
y = data['Outcome']

## <a id='7.4'>7.4. Model Performance</a> 

To measure the performance of a model, we need several elements :

This part is essential

* **Confusion matrix** : also known as the error matrix, allows visualization of the performance of an algorithm :

    * true positive (TP) : Diabetic correctly identified as diabetic
    * true negative (TN) : Healthy correctly identified as healthy
    * false positive (FP) : Healthy incorrectly identified as diabetic
    * false negative (FN) : Diabetic incorrectly identified as healthy

![](https://miro.medium.com/max/2102/1*fxiTNIgOyvAombPJx5KGeA.png)

* **Metrics ** :

    * Accuracy : (TP +TN) / (TP + TN + FP +FN)
    * Precision : TP / (TP + FP)
    * Recall : TP / (TP + FN)
    * F1 score : 2 x ((Precision x Recall) / (Precision + Recall))

* **Roc Curve** : The ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings.

![](https://miro.medium.com/max/722/1*pk05QGzoWhCgRiiFbz-oKQ.png)

* **Precision Recall Curve** :  shows the tradeoff between precision and recall for different threshold
![](https://machinelearningblogcom.files.wordpress.com/2018/04/bildschirmfoto-2018-04-03-um-11-31-16.png)

In [ ]:
def model_performance(model, subtitle) :   
    #Kfold
    cv = KFold(n_splits=5,shuffle=False, random_state = 42)
    y_real = []
    y_proba = []
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0,1,100)
    i = 1
    
    for train,test in cv.split(X,y):
        model.fit(X.iloc[train], y.iloc[train])
        pred_proba = model.predict_proba(X.iloc[test])
        precision, recall, _ = precision_recall_curve(y.iloc[test], pred_proba[:,1])
        y_real.append(y.iloc[test])
        y_proba.append(pred_proba[:,1])
        fpr, tpr, t = roc_curve(y[test], pred_proba[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc) 
    
    # Confusion matrix
    y_pred = cross_val_predict(model, X, y, cv=5)
    conf_matrix = confusion_matrix(y, y_pred)
    trace1 = go.Heatmap(z = conf_matrix  ,x = ["0 (pred)","1 (pred)"],
                        y = ["0 (true)","1 (true)"],xgap = 2, ygap = 2, 
                        colorscale = 'Viridis', showscale  = False)
    
    #Show metrics
    tp = conf_matrix[1,1]
    fn = conf_matrix[1,0]
    fp = conf_matrix[0,1]
    tn = conf_matrix[0,0]
    Accuracy  =  ((tp+tn)/(tp+tn+fp+fn))
    Precision =  (tp/(tp+fp))
    Recall    =  (tp/(tp+fn))
    F1_score  =  (2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn)))))

    show_metrics = pd.DataFrame(data=[[Accuracy , Precision, Recall, F1_score]])
    show_metrics = show_metrics.T

    colors = ['gold', 'lightgreen', 'lightcoral', 'lightskyblue']
    trace2 = go.Bar(x = (show_metrics[0].values), 
                    y = ['Accuracy', 'Precision', 'Recall', 'F1_score'], text = np.round_(show_metrics[0].values,4),
                    textposition = 'auto', textfont=dict(color='black'),
                    orientation = 'h', opacity = 1, marker=dict(
            color=colors,
            line=dict(color='#000000',width=1.5)))

    #Roc curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)

    trace3 = go.Scatter(x=mean_fpr, y=mean_tpr,
                        name = "Roc : " ,
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2), fill='tozeroy')
    trace4 = go.Scatter(x = [0,1],y = [0,1],
                        line = dict(color = ('black'),width = 1.5,
                        dash = 'dot'))
    
    #Precision - recall curve
    y_real = y
    y_proba = np.concatenate(y_proba)
    precision, recall, _ = precision_recall_curve(y_real, y_proba)

    trace5 = go.Scatter(x = recall, y = precision,
                        name = "Precision" + str(precision),
                        line = dict(color = ('lightcoral'),width = 2), fill='tozeroy')
    
    mean_auc=round(mean_auc,3)
    #Subplots
    fig = tls.make_subplots(rows=2, cols=2, print_grid=False,
                          specs=[[{}, {}], 
                                 [{}, {}]],
                          subplot_titles=('Confusion Matrix',
                                          'Metrics',
                                          'ROC curve'+" "+ '('+ str(mean_auc)+')',
                                          'Precision - Recall curve',
                                          ))
    #Trace and layout
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,2,1)
    fig.append_trace(trace4,2,1)
    fig.append_trace(trace5,2,2)
    
    fig['layout'].update(showlegend = False, title = '<b>Model performance report (5 folds)</b><br>'+subtitle,
                        autosize = False, height = 830, width = 830,
                        plot_bgcolor = 'black',
                        paper_bgcolor = 'black',
                        margin = dict(b = 195), font=dict(color='white'))
    fig["layout"]["xaxis1"].update(color = 'white')
    fig["layout"]["yaxis1"].update(color = 'white')
    fig["layout"]["xaxis2"].update((dict(range=[0, 1], color = 'white')))
    fig["layout"]["yaxis2"].update(color = 'white')
    fig["layout"]["xaxis3"].update(dict(title = "false positive rate"), color = 'white')
    fig["layout"]["yaxis3"].update(dict(title = "true positive rate"),color = 'white')
    fig["layout"]["xaxis4"].update(dict(title = "recall"), range = [0,1.05],color = 'white')
    fig["layout"]["yaxis4"].update(dict(title = "precision"), range = [0,1.05],color = 'white')
    for i in fig['layout']['annotations']:
        i['font'] = titlefont=dict(color='white', size = 14)
    py.iplot(fig)

## <a id='7.5'>7.5. Scores Tables</a> 

We can complete model performance report with a table contain all results by fold

In [ ]:
def scores_table(model, subtitle):
    scores = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    res = []
    for sc in scores:
        scores = cross_val_score(model, X, y, cv = 5, scoring = sc)
        res.append(scores)
    df = pd.DataFrame(res).T
    df.loc['mean'] = df.mean()
    df.loc['std'] = df.std()
    df= df.rename(columns={0: 'accuracy', 1:'precision', 2:'recall',3:'f1',4:'roc_auc'})

    trace = go.Table(
        header=dict(values=['<b>Fold', '<b>Accuracy', '<b>Precision', '<b>Recall', '<b>F1 score', '<b>Roc auc'],
                    line = dict(color='#7D7F80'),
                    fill = dict(color='#a1c3d1'),
                    align = ['center'],
                    font = dict(size = 15)),
        cells=dict(values=[('1','2','3','4','5','mean', 'std'),
                           np.round(df['accuracy'],3),
                           np.round(df['precision'],3),
                           np.round(df['recall'],3),
                           np.round(df['f1'],3),
                           np.round(df['roc_auc'],3)],
                   line = dict(color='#7D7F80'),
                   fill = dict(color='#EDFAFF'),
                   align = ['center'], font = dict(size = 15)))

    layout = dict(width=800, height=400, title = '<b>Cross Validation - 5 folds</b><br>'+subtitle, font = dict(size = 15))
    fig = dict(data=[trace], layout=layout)

    py.iplot(fig, filename = 'styled_table')

# <a id='8'>8. Machine Learning</a> 

In [ ]:
# Load libraries
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

## <a id='8.1'>8.1 Train Test Split</a> 

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.25, random_state=42, stratify=data['Outcome'])

In [ ]:
def GetBasedModel():
    basedModels = []
    basedModels.append(('LR'   , LogisticRegression()))
    basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
    basedModels.append(('KNN'  , KNeighborsClassifier()))
    basedModels.append(('CART' , DecisionTreeClassifier()))
    basedModels.append(('NB'   , GaussianNB()))
    basedModels.append(('SVM'  , SVC(probability=True)))
    basedModels.append(('AB'   , AdaBoostClassifier()))
    basedModels.append(('GBM'  , GradientBoostingClassifier()))
    basedModels.append(('RF'   , RandomForestClassifier()))
    basedModels.append(('ET'   , ExtraTreesClassifier()))

    
    return basedModels

In [ ]:
models = GetBasedModel()

for name, model in models:
    clf = model.fit(X, y)
    model_performance(clf, name)
    scores_table(clf, name)
    
    
